In [2]:
from os import listdir
from PIL import Image
from torchvision import transforms
import torch
import json
import random
import torch.optim as optim
import torch.nn as nn

In [3]:
class YOLOv1(nn.Module):
    """
    This class contains the YOLOv1 model. It consists of 24 convolutional and
    2 fully-connected layers which divide the input image into a 
    split_size x split_size grid and predict num_boxes bounding boxes per grid
    cell. If the confidence of a bounding box reaches a certain value, it is 
    considered as a valid prediction.
    """
    
    def __init__(self, split_size, num_boxes, num_classes):
        """
        Initializes the neural-net with the parameter values to produce the
        desired predictions.
        
        Parameters:
            split_size (int): Size of the grid which is applied to the image.
            num_boxes (int): Amount of bounding boxes which are predicted per 
                grid cell.
            num_classes (int): Amount of different classes which are being 
                predicted by the model.
        """
        
        super(YOLOv1, self).__init__()
        self.darkNet = nn.Sequential(
            nn.Conv2d(3, 64, 7, padding=3, stride=2, bias=False),
                nn.BatchNorm2d(64),
                nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 192, 3, padding=1, bias=False),
                nn.BatchNorm2d(192),
                nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(192, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.1),
            nn.Conv2d(128, 256, 3, padding=1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, 1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, padding=1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(512, 256, 1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, padding=1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, 1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, padding=1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, 1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, padding=1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, 1, bias=False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, padding=1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.Conv2d(512, 512, 1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, 3, padding=1, bias=False),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(1024, 512, 1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, 3, padding=1, bias=False),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 512, 1, bias=False),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, 3, padding=1, bias=False),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, 3, padding=1, bias=False),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, 3, padding=1, stride=2, bias=False),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1),
            
            nn.Conv2d(1024, 1024, 3, padding=1, bias=False),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, 3, padding=1, bias=False),
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1),
            )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * split_size * split_size, 4096),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, split_size * split_size * (num_classes + num_boxes*5))
            )
        
    def forward(self, x):
        """
        Forwards the input tensor through the model to produce the predictions. 
        Parameters:
            x (tensor): A tensor of shape (batch_size, 3, 448, 448) which represents
                a batch of images.
        """
        x = self.darkNet(x)
        x = self.fc(x)
        return x

In [4]:
class DataLoader():
    """
    This class uses its attributes to load the training data and transforms it into tensors.
    The tensors are then stored in mini-batches inside the train_data list which is the final 
    product of this class. Multiple function calls of LoadData() will initialize the train_data 
    list with new tensors from the training data, excluding all the previous ones. 
    """

    def __init__(self, train_files_path, target_files_path, category_list, split_size, batch_size, train_size):
        """
        Initialize all parameters for loading and transforming the data into tensors.
        
        Parameters:
            train_files_path (string): The path to the train image folder
            target_files_path (string): The path to the json file containg the image labels
            category_list (list): Reference list to all the label categories for object detection
            split_size (int): Amount of grid cells
            batch_size (int): Batch size
            train_size (int): Amount of images which are loaded as training data for one epoch
        """
        
        self.train_files_path = train_files_path
        self.target_files_path = target_files_path       
        self.category_list = category_list        
        self.num_classes = len(category_list)       
        self.cells = split_size        
        self.batch_size = batch_size      
        self.train_size = train_size
        
        self.train_files = [] # Will contain the remaining image names from the folder
        self.target_files = [] # Will contain the json elements with the ground-truth labels
        
        self.train_data = [] # Will contain tuples with mini-batches of image and label tensors    
        self.img_tensors = [] # Used to temporary store samples from a single batch
        self.target_tensors = [] # Used to temporary store samples from a single batch
        
        # Define transform which is applied to every single image to resize and convert it into a tensor
        self.transform = transforms.Compose([
            transforms.Resize((448,448), Image.NEAREST),
            transforms.ToTensor(),
            ])
    

    def LoadFiles(self):
        """
        First function to be executed.
        Loads the images and the label file using the respective system path.
        """
            
        # All image names from the directory are loaded into the list train_files.
        self.train_files = listdir(self.train_files_path)
        
        # The json file containing the labels is loaded into the list target_files.
        f = open(self.target_files_path)
        self.target_files = json.load(f)
        
        
    def LoadData(self):
        """
        Transforms the training images and labels into tensors and loads them into batches. Once a batch is
        full, it is stored in the train_data list. Fills the train_data list with batches until the desired
        train_size is reached. Every image that is loaded, is being excluded from future calls of this 
        function.
        """
        
        # Reset the cache
        self.train_data = []    
        self.img_tensors = [] 
        self.target_tensors = [] 

        for i in range(len(self.train_files)):
            # Check if batch is full and perhaps start a new one
            if len(self.img_tensors) >= self.batch_size:
                self.train_data.append((torch.stack(self.img_tensors), torch.stack(self.target_tensors)))
                self.img_tensors = []
                self.target_tensors = []
                print('Loaded batch ', len(self.train_data), 'of ', int(self.train_size/self.batch_size))
                print('Percentage Done: ', round(len(self.train_data)/int(self.train_size/self.batch_size)*100., 2), '%')
                print('')
            
            if i == self.train_size:
                break # The train_data list is full with the desired amount of batches
                
            # Extracts a single random image and the corresponding label, and transforms them into
            # tensors. Both are appended to the img_tensors and target_tensors lists
            self.extract_image_and_label() 


    def extract_image_and_label(self):
        """
        Chooses a random image which is then being transformed into a tensor and stored.
        Finds the corresponding label inside the json file which is then being transformed into a tensor
        and stored. Stores both tensors inside the img_tensors and target_tensors lists.
        """
        
        img_tensor, chosen_image = self.extract_image()
        target_tensor = self.extract_json_label(chosen_image)

        self.img_tensors.append(img_tensor)
        self.target_tensors.append(target_tensor)

        
    def extract_image(self):   
        """
        Finds a random image from the train_files list and applies the transform to it. 
 
        Returns:
            img_tensor (tensor): The tensor which contains the image values
            f (string): The string name of the image file
        """    
        
        f = random.choice(self.train_files)
        self.train_files.remove(f)
        global img
        img = Image.open(self.train_files_path + f)
        img_tensor = self.transform(img) # Apply the transform to the image.
        return img_tensor, f


    def extract_json_label(self, chosen_image):
        """
        Uses the name of the image to find the corresponding json element. Then it extracts the data and
        transforms it into a tensor which is stored inside the target_tensors list.

        Parameters:
            chosen_image (string): The name of the image for which the label is needed.

        Returns:
            target_tensor (tensor): The tensor which contains the image labels
        """
        
        for json in self.target_files:
            if json['name'] == chosen_image:
                img_label = json
                break

        target_tensor = self.transform_label_to_tensor(img_label)
        return target_tensor


    def transform_label_to_tensor(self, img_label):
        """
        Extracts the useful information from the json element and transforms them into a tensor.
        
        Parameters:
            img_label (): A specific json element
            
        Returns:
            target_tensor (tensor): A tensor of size (5+num_classes,cells,cells) which is used as the target of 
            the image.
        """
        
        target_tensor = torch.zeros(5+self.num_classes, self.cells, self.cells) # Here are the information stored

        for labels in range(len(img_label["labels"])):

            # Store the category index if its contained within the category_list.
            category = img_label["labels"][labels]["category"]         
            if category not in self.category_list:
                continue
            ctg_idx = self.category_list.index(category)

            # Store the bounding box information and rescale it by the resize factor.
            x1 = img_label["labels"][labels]["box2d"]["x1"] * (448/img.size[0])
            y1 = img_label["labels"][labels]["box2d"]["y1"] * (448/img.size[1])
            x2 = img_label["labels"][labels]["box2d"]["x2"] * (448/img.size[0])
            y2 = img_label["labels"][labels]["box2d"]["y2"] * (448/img.size[1])

            # Transforms the corner bounding box information into a mid bounding box information
            x_mid = abs(x2 - x1) / 2 + x1
            y_mid = abs(y2 - y1) / 2 + y1
            width = abs(x2 - x1) 
            height = abs(y2 - y1) 

            # Size of a single cell
            cell_dim = int(448 / self.cells)

            # Determines the cell position of the bounding box
            cell_pos_x = int(x_mid // cell_dim)
            cell_pos_y = int(y_mid // cell_dim)

            # Stores the information inside the target_tensor
            if target_tensor[0][cell_pos_y][cell_pos_x] == 1: # Check if the cell already contains an object
                continue
            target_tensor[0][cell_pos_y][cell_pos_x] = 1
            target_tensor[1][cell_pos_y][cell_pos_x] = (x_mid % cell_dim) / cell_dim
            target_tensor[2][cell_pos_y][cell_pos_x] = (y_mid % cell_dim) / cell_dim
            target_tensor[3][cell_pos_y][cell_pos_x] = width / 448
            target_tensor[4][cell_pos_y][cell_pos_x] = height / 448
            target_tensor[ctg_idx+5][cell_pos_y][cell_pos_x] = 1

        return target_tensor

In [8]:
def TrainNetwork(num_epochs):
    """
    Starts the training process of the model.
    Parameters:
        num_epochs (int): Amount of epochs for training the model
    """
    
    data = DataLoader(train_files_path, target_files_path, category_list, split_size, batch_size, train_size)
    
    for epoch in range(num_epochs):
        model.train()
        
        print("DATA IS BEING LOADED FOR A NEW EPOCH")
        print("")
        data.LoadFiles()
        
        while len(data.train_files) > 0:
            print("LOADING NEW BATCHES")            
            print("Remaining files:" + str(len(data.train_files)))
            print("")
            data.LoadData()
            
            for batch_idx, (train_data, target_data) in enumerate(data.train_data):
                train_data = train_data.to(device)
                target_data = target_data.to(device)
    
                predictions = model(train_data)
                print(predictions.shape)
                """
                loss = YOLO_Loss(predictions, target_data, split_size, num_boxes, num_classes, lambda_coord, lambda_noobj)

                optimizer.zero_grad()
                loss.backward()

                optimizer.step()

                print('Train Epoch: {} of {} [Batch: {}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, num_epochs, batch_idx, len(data.train_data),
                    batch_idx / len(train_data) * 100., loss.data.item()))
                """
                print('Train Epoch: {} of {} [Batch: {}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch+1, num_epochs, batch_idx+1, len(data.train_data),
                    (batch_idx+1) / len(data.train_data) * 100., 10))
                print('')

# Dataset parameters
train_files_path = "C:/Users/alens/Desktop/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/YOLO v1/bdd100k/images/100k/val/"
target_files_path = "C:/Users/alens/Desktop/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/YOLO v1/bdd100k_labels_release/bdd100k/labels/det_v2_val_release.json"
category_list = ["other vehicle", "pedestrian", "traffic light", "traffic sign", "truck", "train", "other person", "bus", "car", "rider", "motorcycle", "bicycle", "trailer"]


# Hyperparameters
learning_rate = 0.001
split_size = 7
num_boxes = 2
num_classes = len(category_list)
lambda_coord = 5
lambda_noobj = 5
batch_size = 64
num_epochs = 10
train_size = 500


# Set device
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
devie = 'cpu'

# Initialize model
model = YOLOv1(split_size, num_boxes, num_classes).to(device)

# Define the learning method as stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Start the training process
TrainNetwork(num_epochs)

RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 8.00 GiB total capacity; 6.11 GiB already allocated; 222.24 MiB free; 6.16 GiB reserved in total by PyTorch)